# Jagdeesh & Titman

In [ ]:
%pip install --upgrade fyers_helper TA-Lib

In [1]:
root_path = '/content/drive/MyDrive/MomentumChasers'
data_path = f'{root_path}/data'
config_path = f'{root_path}/fyers-api-creds.json'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/NIFTY_50'
tables = pd.read_html(url)
nifty50_df = tables[1]

url = 'https://en.wikipedia.org/wiki/NIFTY_Next_50'
tables = pd.read_html(url)
niftyNext50_df = tables[2]

tickers = pd.concat([nifty50_df[["Symbol"]], niftyNext50_df[["Symbol"]]], axis=0).reset_index(drop=True)
tickers = [f"NSE:{t}-EQ" for t in tickers['Symbol'].to_list()]

In [4]:
from fyers_helper import prepare_data, load_stock_data
import datetime as dt
import os
from google.colab import userdata

end_date = dt.datetime.now()
start_date = dt.datetime(2015, 1, 1)

os.environ['FYERS_CONFIG'] = userdata.get('FYERS_CONFIG')

file_paths = prepare_data(tickers, "1D", start_date=start_date, end_date=end_date, path=data_path, overwrite=False)
loaded_data = load_stock_data(file_paths, data_path, "1D")

Loading 1Dmin data: 100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


In [19]:
df = pd.concat(loaded_data, names=["Stock", "Date"]).reset_index()

In [20]:
import talib as ta

# Calculate RSI per stock
df['RSI'] = (
    df.groupby('Stock')['Close']
      .transform(lambda x: ta.RSI(x, timeperiod=14))
)
df['Entry_Price'] = df.groupby('Stock')['Close'].shift(-1)
df['Entry_Time'] = df.groupby('Stock')['Date'].shift(-1)

# df = df[df.Date >= '2024-01-01 05:30:00']

In [21]:
signals = (
    df[df.RSI < 35].groupby('Date', group_keys=False)
    .apply(lambda x: x.nsmallest(5, "RSI"))
    .sort_values('Date').reset_index(drop=True)
)

/tmp/ipython-input-3493244206.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nsmallest(5, "RSI"))


In [31]:
from dataclasses import dataclass, field
from typing import List, Optional
from tqdm.notebook import tqdm

leverage = 1
initial_capital = 200000
max_positions_num = 40
capital_buckets = [initial_capital/max_positions_num] * max_positions_num
sl_perc = 20
tp_perc = 6.28
max_holding_days = 10
active_positions = {}
closed_positions = []
rsi_points_for_averaging = [35, 30, 25, 20, 15, 10, 5]


@dataclass
class Trade:
  entry_time: str
  entry_price: float
  quantity: int
  rsi: float

@dataclass
class Position:
  stock: str
  entry_time: str
  avg_entry_price: float = 0.0
  quantity: int = 0
  exit_time: str = None
  exit_price: float = 0.0
  sl: float = 0.0
  tp: float = 0.0
  rsi: float = 0.0
  trades: List[Trade] = field(default_factory=list)
  pnl: float = 0.0
  tax: float = 0.0

  def exit_margin(self):
    return (self.avg_entry_price * self.quantity)/leverage - self.tax + self.pnl

  def close(self, exit_time, exit_price):
    self.exit_time = exit_time
    self.exit_price = exit_price
    self.pnl = (exit_price - self.avg_entry_price) * self.quantity
    self.calculate_taxes()

  def calculate_taxes(self):
    extry_taxes = 0
    for trade in self.trades:
      stt = abs(trade.quantity) * trade.entry_price * 0.025/100
      transaction_charges = (abs(trade.quantity) * trade.entry_price * 0.00322/100)
      gst = (stt + transaction_charges) * 18/100
      extry_taxes += stt + transaction_charges + gst

    transaction_charges = abs(self.quantity) * self.exit_price * 0.00322/100
    gst = transaction_charges * 18/100
    stamp_duty = abs(self.quantity) * self.exit_price * 0.003/100
    exit_taxes = transaction_charges + gst + stamp_duty

    self.tax = extry_taxes + exit_taxes

  def next_rsi_threshold(self):
    return [r for r in rsi_points_for_averaging if r < self.rsi][0]

  def rebalance_position(self):
    total_cost = 0
    total_qty = 0
    for t in self.trades:
      total_cost += t.entry_price * t.quantity
      total_qty += t.quantity
    self.avg_entry_price = total_cost / total_qty
    self.quantity = total_qty
    self.sl = self.avg_entry_price * (1 - sl_perc / 100)
    self.tp = self.avg_entry_price * (1 + tp_perc / 100)

  def add_trade(self, trade: Trade):
    self.trades.append(trade)
    self.rebalance_position()
    self.rsi = trade.rsi


def select_stock_for_entry(_df: pd.DataFrame, capital):
  for _, row in _df.iterrows():
    # print(f'{row.Stock} - {row.Stock not in active_positions} - {row.Entry_Price}')
    if active_positions.get(row.Stock, None) is None and row.Entry_Price <= capital:
      return row

def get_capital():
  if len(capital_buckets) == 0:
    return None
  return capital_buckets.pop()

def has_capital():
  return len(capital_buckets) > 0

def update_capital_buckets(c = 0):
  global capital_buckets
  if len(capital_buckets) > 0:
    capital_buckets = [(sum(capital_buckets) + c)/len(capital_buckets)] * len(capital_buckets)

def init_new_position(selected_stock, capital: float):
  if selected_stock is not None:
    qty = int(capital * leverage / selected_stock.Entry_Price)
    position = Position(selected_stock.Stock, selected_stock.Entry_Time)
    trade = Trade(selected_stock.Entry_Time, selected_stock.Entry_Price, qty, selected_stock.RSI)
    position.add_trade(trade)

    remaining_capital = (capital - qty * trade.entry_price) / leverage
    update_capital_buckets(remaining_capital)
    return position

def rebalance_active_positions(k, v):
  global active_positions, capital_buckets
  for key in active_positions.keys():
    position = active_positions.get(key, None)
    if position is None:
      continue

    rows = v[v.Stock == position.stock]
    if len(rows) == 0:
      continue
    row = rows.iloc[0]
    if row.RSI < position.next_rsi_threshold():
      capital = get_capital()
      if capital is not None:
        qty = position.trades[0].quantity
        trade = Trade(row.Entry_Time, row.Entry_Price, qty, row.RSI)
        position.add_trade(trade)

        remaining_capital = ((capital * leverage) - (qty * trade.entry_price)) / leverage
        update_capital_buckets(remaining_capital)

    elif row.High >= position.tp or row.Low <= position.sl:
      exit_price = position.tp if row.High >= position.tp else position.sl
      position.close(row.Date, exit_price)
      closed_positions.append(position)
      active_positions[position.stock] = None
      capital_buckets += [position.exit_margin() / len(position.trades)] * len(position.trades)
      update_capital_buckets()
    else:
      holding_days = df[(df.Stock == position.stock) & (position.entry_time <= df.Date) & (k >= df.Date)]
      if len(holding_days) == max_holding_days:
        position.close(row.Date, row.Close)
        closed_positions.append(position)
        active_positions[position.stock] = None
        capital_buckets += [position.exit_margin() / len(position.trades)] * len(position.trades)
        update_capital_buckets()

no_stocks_found_days = 0

def backtest():
  pb = tqdm(total=len(df.Date.unique()), desc="Backtesting")
  for k, v in df.groupby('Date'):
    rebalance_active_positions(k, v)

    global active_positions, capital_buckets

    if has_capital():
      capital = get_capital()
      selected_stock = select_stock_for_entry(signals[signals.Date == k], capital)
      if selected_stock is not None:
        position = init_new_position(selected_stock, capital)
        active_positions[position.stock] = position
      else:
        capital_buckets.append(capital)
        global no_stocks_found_days
        no_stocks_found_days += 1
    #     print("found no stock", k, len(signals[signals.Date == k]), capital, len(capital_buckets))
    # else:
    #   print("Out of money :(")

    pb.update(1)
  pb.close()



backtest()


Backtesting:   0%|          | 0/2630 [00:00<?, ?it/s]

In [32]:
from dataclasses import  asdict
pd.DataFrame([asdict(p) for p in closed_positions]).sort_values(['entry_time']).to_csv(f'{root_path}/closed_positions.csv', index=False)


In [10]:
[len(p.trades) for p in active_positions.values() if p is not None]

[]

In [46]:
v = df[df.Date == '2025-07-29 05:30:00']

apdf = pd.DataFrame(closed_positions).set_index('stock').join(v.set_index('Stock')[['Close', 'Entry_Time']])
apdf['perc_change'] = (apdf.Close - apdf.avg_entry_price)/apdf.avg_entry_price * 100
apdf.sort_values('perc_change').iloc[0]

,NSE:ADANIENSOL-EQ
entry_time,2022-11-23 05:30:00
avg_entry_price,2856.85
quantity,2
exit_time,2022-12-06 05:30:00
exit_price,2773.0
sl,2285.48
tp,3036.26018
rsi,24.864723
trades,"[{'entry_time': 2022-11-23 05:30:00, 'entry_pr..."
pnl,-167.7


In [151]:
signals.to_csv(f'{root_path}/signals.csv', index=False)

In [154]:
df[df.Date >= '2024-01-01 05:30:00'].groupby('Stock').count()

,Date,Open,High,Low,Close,Volume,RSI,Entry_Price,Entry_Time
Stock,,,,,,,,,
NSE:ABB-EQ,402,402,402,402,402,402,402,401,401
NSE:ADANIENSOL-EQ,402,402,402,402,402,402,402,401,401
NSE:ADANIENT-EQ,402,402,402,402,402,402,402,401,401
NSE:ADANIGREEN-EQ,402,402,402,402,402,402,402,401,401
NSE:ADANIPORTS-EQ,402,402,402,402,402,402,402,401,401
...,...,...,...,...,...,...,...,...,...
NSE:UNITDSPR-EQ,401,401,401,401,401,401,401,400,400
NSE:VBL-EQ,402,402,402,402,402,402,402,401,401
NSE:VEDL-EQ,402,402,402,402,402,402,402,401,401
